[![MIT licensed](https://img.shields.io/badge/license-MIT-blue.svg)](https://opensource.org/licenses/MIT)

# rocPyDecode 
##### rocDecode Python Binding

> **NOTE**
> The published documentation is available at [rocPyDecode](https://rocm.docs.amd.com/projects/rocPyDecode/en/latest/index.html) in an organized, easy-to-read format, with search and a table of contents. The documentation source files reside in the `docs` folder of this repository. As with all ROCm projects, the documentation is open source. For more information on contributing to the documentation, see [Contribute to ROCm documentation](https://rocm.docs.amd.com/en/latest/contribute/contributing.html).


The rocDecode Python Binding, rocPyDecode, is a tool that allows users to access rocDecode APIs in both Python and C/C++ languages. It works by connecting Python and C/C++ libraries, enabling function calling and data passing between the two languages. The rocpydecode.so library is a wrapper that facilitates the use of rocDecode APIs that are written primarily in C/C++ language within Python.

## Prerequisites

* Linux distribution
  * Ubuntu - `20.04` / `22.04`

* [ROCm-supported hardware](https://rocm.docs.amd.com/projects/install-on-linux/en/latest/reference/system-requirements.html)
> **IMPORTANT**
> `gfx908` or higher GPU required

* Install ROCm `6.2.0` or later with [amdgpu-install](https://rocm.docs.amd.com/projects/install-on-linux/en/latest/how-to/amdgpu-install.html): Required usecase - rocm
> **IMPORTANT**
> `sudo amdgpu-install --usecase=rocm`

# Video Decode Python Sample

### Import rocPyDecode Modules

In [ ]:
import pyRocVideoDecode.decoder as dec
import pyRocVideoDecode.demuxer as dmx

##### Other needed modules

In [ ]:
import datetime
import sys
import argparse
import os.path

### Construct Demuxer & Decoder Instances 

<i>Set parameters to construct demuxer and decoder instance</i>

In [ ]:
input_file_path  = "/opt/rocm/share/rocdecode/video/AMD_driving_virtual_20-H265.mp4" # change accordingly to set to valid and existing video file
output_file_path = None
device_id = 0
mem_type = 0 
b_force_zero_latency = False
crop_rect = None
b_generate_md5 = False
ref_md5_file = None
seek_frame = -1
seek_mode = 1
seek_criteria = 0
resize_dim = None

#### Instantiate demuxer and decoder

In [ ]:
# demuxer instance
demuxer = dmx.demuxer(input_file_path)

# get the used coded id
codec_id = dec.GetRocDecCodecID(demuxer.GetCodecId())

# decoder instance
viddec = dec.decoder(
    codec_id,
    device_id,
    mem_type,
    b_force_zero_latency,
    crop_rect,
    0,
    0,
    1000)

<i>Check if codec of the input video is supported</i>

In [ ]:
# Get GPU device information
cfg = viddec.GetGpuInfo()

# check if codec is supported
if (viddec.IsCodecSupported(device_id, codec_id, demuxer.GetBitDepth()) == False):
    print("ERROR: Codec is not supported on this GPU " + cfg.device_name)
    exit()

<i>Printout system/env information</i>

In [ ]:
#  print some GPU info out
print("\ninfo: Input file: " + 
      input_file_path + '\n' + "info: Using GPU device " + str(device_id) + " - " +
      cfg.device_name + "["  + cfg.gcn_arch_name + "] on PCI bus " + str(cfg.pci_bus_id) +
      ":" + str(cfg.pci_domain_id) + "." + str(cfg.pci_device_id))

Setup MD5 if requested

In [ ]:
# md5 file full path & md5 flag
b_md5_check = False
if (ref_md5_file is not None):
    if os.path.exists(ref_md5_file):
        b_generate_md5 = True
        b_md5_check = True

# init MD5 if requested
if b_generate_md5:
    viddec.InitMd5()

<i> Set reconfiguration params</i>

In [ ]:
# set reconfiguration params based on user arguments
flush_mode = 0
if (output_file_path is not None):
    flush_mode = 1
elif b_generate_md5:
    flush_mode = 2
viddec.SetReconfigParams(flush_mode, output_file_path if (output_file_path is not None) else str(""))

## Start the decoding loop

In [ ]:
# Setup vars for the decoding loop
n_frame = 0
total_dec_time = 0.0
frame_is_resized = False
not_seeking = True if (seek_frame == -1) else False
session_id = 0
if (resize_dim is not None):
    resize_dim = None if(resize_dim[0] == 0 or resize_dim[1] == 0) else resize_dim

# start decoding    
print("info: decoding started, please wait! \n")   
    
while True:
    start_time = datetime.datetime.now()
    if(not_seeking):
        packet = demuxer.DemuxFrame()
    else:
        packet = demuxer.SeekFrame(seek_frame, seek_mode, seek_criteria)
        not_seeking = True
    n_frame_returned = viddec.DecodeFrame(packet)
    for i in range(n_frame_returned):
        viddec.GetFrameYuv(packet)
        if (b_generate_md5):
            surface_info = viddec.GetOutputSurfaceInfo()
            viddec.UpdateMd5ForFrame(packet.frame_adrs, surface_info)
        if (resize_dim is not None):
            surface_info = viddec.GetOutputSurfaceInfo()
            if(viddec.ResizeFrame(packet, resize_dim, surface_info) != 0):
                frame_is_resized = True
            else:
                frame_is_resized = False
        if (output_file_path is not None):
            if (frame_is_resized):
                resized_surface_info = viddec.GetResizedOutputSurfaceInfo()
                viddec.SaveFrameToFile(output_file_path, packet.frame_adrs_resized, resized_surface_info)
            else:
                viddec.SaveFrameToFile(output_file_path, packet.frame_adrs)

        # release frame
        viddec.ReleaseFrame(packet)

    # measure after completing a whole frame
    end_time = datetime.datetime.now()
    time_per_frame = end_time - start_time
    total_dec_time = total_dec_time + time_per_frame.total_seconds()

    # increament frames counter
    n_frame += n_frame_returned
    if (packet.bitstream_size <= 0):  # EOF: no more to decode
        break
        
# decoding end
print("info: decoding ended! \n")           

### Beyond the decoding loop

<i>Calculate overhead and FPS</i>

In [ ]:
# after the decoding loop
n_frame += viddec.GetNumOfFlushedFrames()
print("info: Total frame decoded: " + str(n_frame))

# Calculate if overhead
if (output_file_path is None):
    if (n_frame > 0 and total_dec_time > 0):
        time_per_frame = (total_dec_time / n_frame) * 1000
        session_overhead = viddec.GetDecoderSessionOverHead(session_id)
        if (session_overhead == None):
            session_overhead = 0
        time_per_frame -= (session_overhead / n_frame) # remove the overhead
        frame_per_second = n_frame / total_dec_time
        print("info: avg decoding time per frame: " +"{0:0.2f}".format(round(time_per_frame, 2)) + " ms")
        print("info: avg frame per second: " +"{0:0.2f}".format(round(frame_per_second,2)) +"\n")
    else:
        print("info: frame count= ", n_frame)

<i> MD5-Digest if requested</i>

In [ ]:
# if MD5 check requested
if b_generate_md5:
    digest = viddec.FinalizeMd5()
    print("MD5 message digest: ", end=" ")
    str_digest = ""
    for i in range(16):
        str_digest = str_digest + str(format('%02x' % int(digest[i])))
    print(str_digest)
    if (b_md5_check):
        f = open(ref_md5_file)
        md5_from_file = f.read(16 * 2)
        b_match = (md5_from_file == str_digest)
        if (b_match):
            print("MD5 digest matches the reference MD5 digest.\n")
        else:
            print(
                "MD5 digest does not match the reference MD5 digest: ",
                md5_from_file)